# Import packages

In [1]:
import os
import cmaps
import cmocean
import numpy as np
import xarray as xr
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
from scipy.interpolate import griddata
from netCDF4 import Dataset
from datetime import date
from xarrayMannKendall import Mann_Kendall_test
from mpl_toolkits.basemap import Basemap
from matplotlib.pyplot import Polygon
from matplotlib import rcParams
from matplotlib.backends.backend_pdf import PdfPages
rcParams['font.family'] = 'sans-serif'
directory   = '/g/data/fu5/eac/20year_run_Junde/jl4424/KC_GS'

# Read BRAN meridional volume and heat transport

In [2]:
#################################################################################################
dataset0         = sio.loadmat(directory+'/Figures/Figure2_mean.mat')
bran_lon         = dataset0['bran_lon'][:,:]
bran_lat         = dataset0['bran_lat'][:,:]
bran_v_transport = dataset0['bran_v_transport_mean'][:,:]
KC_u_transport   = dataset0['bran_u_transport_mean_KC'][:,:] # KC
GS_u_transport   = dataset0['bran_u_transport_mean_GS'][:,:] # GS
KC_v_transport   = dataset0['bran_v_transport_mean_KC'][:,:] # KC
GS_v_transport   = dataset0['bran_v_transport_mean_GS'][:,:] # GS
KC_lon           = bran_lon[200:502,299:451]                  # KC
GS_lon           = bran_lon[1600:1902,299:451]                # GS
KC_lat           = bran_lat[200:502,299:451]                  # KC
GS_lat           = bran_lat[1600:1902,299:451]                # GS
#################################################################################################
dataset1  = xr.open_dataset(directory+'/BRAN/trends/transports/BRAN_u_transport_trends_KC.nc')
dataset2  = xr.open_dataset(directory+'/BRAN/trends/transports/BRAN_u_transport_trends_GS.nc')
dataset3  = xr.open_dataset(directory+'/BRAN/trends/transports/BRAN_v_transport_trends_KC.nc')
dataset4  = xr.open_dataset(directory+'/BRAN/trends/transports/BRAN_v_transport_trends_GS.nc')
KC_u_transport_trends = dataset1.trend.transpose().values*120
KC_u_transport_p = dataset1.p.transpose().values
KC_u_transport_p[np.where(KC_u_transport_p > 0.05)] = np.nan
GS_u_transport_trends = dataset2.trend.transpose().values*120
GS_u_transport_p = dataset2.p.transpose().values
GS_u_transport_p[np.where(GS_u_transport_p > 0.05)] = np.nan
KC_v_transport_trends = dataset3.trend.transpose().values*120
KC_v_transport_p = dataset3.p.transpose().values
KC_v_transport_p[np.where(KC_v_transport_p > 0.05)] = np.nan
GS_v_transport_trends = dataset4.trend.transpose().values*120
GS_v_transport_p = dataset4.p.transpose().values
GS_v_transport_p[np.where(GS_v_transport_p > 0.05)] = np.nan
#################################################################################################
dataset5 = xr.open_dataset(directory+'/BRAN/BRAN2020_KC_GS_clim_SSH.nc')
bran_ssh = dataset5.clim_ssh.transpose()
KC_ssh   = bran_ssh[200:502,299:451]
GS_ssh   = bran_ssh[1600:1902,299:451]

# Plot the trends of spatial distribution of mean zonal and meridional volume transport 

In [ ]:
##############################################################################################################
labels        = ['a','b','c','d']
levels1       = np.linspace(-1,1,50)
tick_marks1   = np.linspace(-1,1,5)
cmaps1        = cmocean.cm.balance
fig_ratio     = 0.9
fig           = plt.figure(figsize=(24, 24))
gs            = gridspec.GridSpec(2,2)
labelfont     = 25
padspacescale = 10
labelpadscale = 2
linefont      = 3
scale1        = 1.0
scale2        = 1.0
m_scale       = 50   
##############################################################################################################
for i in range(4):
    ax = fig.add_subplot(gs[i])
    l, b, w, h = ax.get_position().bounds
    if i==0:  # KC
        lon        = KC_lon      
        lat        = KC_lat
        var_trends = KC_u_transport_trends 
        var_mean   = KC_u_transport
        var_p      = KC_u_transport_p
    elif i==1: # GS
        lon        = GS_lon   
        lat        = GS_lat
        var_trends = GS_u_transport_trends  
        var_mean   = GS_u_transport
        var_p      = GS_u_transport_p
    if i==2: # KC
        b          = b+0.15
        lon        = KC_lon      
        lat        = KC_lat
        var_trends = KC_v_transport_trends 
        var_mean   = KC_v_transport
        var_p      = KC_v_transport_p
    elif i==3: # GS
        b          = b+0.15
        lon        = GS_lon   
        lat        = GS_lat
        var_trends = GS_v_transport_trends  
        var_mean   = GS_v_transport
        var_p      = GS_v_transport_p

    m           = Basemap(projection='merc',suppress_ticks=True,llcrnrlat=np.nanmin(lat),urcrnrlat=np.nanmax(lat),llcrnrlon=np.nanmin(lon),urcrnrlon=np.nanmax(lon),resolution='c')        
    x, y        = m(lon, lat)
    yy          = np.arange(1, y.shape[0], 5)
    xx          = np.arange(1, x.shape[1], 5)
    points      = np.meshgrid(yy, xx)
    point_index = tuple(points)
    point_x     = x[point_index]
    point_y     = y[point_index]
    point_z     = var_p[point_index]
    point_x     = np.ravel(point_x)
    point_y     = np.ravel(point_y)
    point_z     = np.ravel(point_z)
    point_x     = point_x[~np.isnan(point_z)]
    point_y     = point_y[~np.isnan(point_z)]

    if i==0:
        plt.ylabel(r'Latitude($^\circ$)',fontsize=labelfont,labelpad=80,family='sans-serif') 
    elif i==2:
        plt.xlabel(r'Longitude($^\circ$)',fontsize=labelfont,labelpad=30,family='sans-serif') 
        plt.ylabel(r'Latitude($^\circ$)',fontsize=labelfont,labelpad=80,family='sans-serif') 
    elif i==3:
        plt.xlabel(r'Longitude($^\circ$)',fontsize=labelfont,labelpad=30,family='sans-serif') 
    CB1=m.contourf(x, y, var_trends,cmap=cmaps1,levels=levels1,origin='lower',extend='both')
    CS = m.scatter(point_x,point_y,s=m_scale,c='dimgray',marker='.')         
    CS1=m.contour(x, y, var_mean,levels=np.array([0.5, 1.0, 2.0, 3.0]),linewidths=1.5*linefont,colors='tab:orange',linestyles='solid')
    CS11=m.contour(x, y, var_mean,levels=np.array([-3.0,-2.0,-1.0,-0.5]),linewidths=1.5*linefont,colors='tab:purple',linestyles='dashed')
    # CS111=m.contour(x, y, var_mean,levels=[0.0],linewidths=linefont,colors='black',linestyles='dashed')
    labels1=plt.clabel(CS1, inline=True, fontsize=20, inline_spacing=5, fmt="%.1f") 
    labels11=plt.clabel(CS11, inline=True, fontsize=20,inline_spacing=5, fmt="%.1f")  
    for label1 in labels1:
        label1.set_color('black')  
        label1.set_fontweight('bold') 
    for label11 in labels11:
        label11.set_color('black')  
        label11.set_fontweight('bold')  
    if np.mod(i,2)==0:
        cx,cy = m(135, 46.0)
        ax.set_position([l, b, scale1*w, scale1*h])   
        CS2 = m.contour(x, y, KC_ssh, [0.7],linewidths=2*linefont,linestyles='solid',colors='tab:gray')
    elif np.mod(i,2)==1:
        cx,cy  = m(275, 46.0)
        ax.set_position([l, b, scale2*w, scale2*h])
        CS2 = m.contour(x, y, GS_ssh, [0.0],linewidths=2*linefont,linestyles='solid',colors='tab:blue')
        
    m.drawparallels(np.arange(15,50, 5),labels=[1,0,0,0],linewidth=linefont,dashes=[2,2],color='.5',fontsize=labelfont,family='sans-serif')    
    m.drawmeridians(np.arange(0,360, 5),labels=[0,0,0,1],linewidth=linefont,dashes=[2,2],color='.5',fontsize=labelfont,family='sans-serif')
    m.drawcoastlines(color='0.1',  linewidth=linefont)
    m.fillcontinents(color='0.9', lake_color='white') 
    plt.text(cx, cy,labels[i],color='xkcd:black', fontsize=labelfont, weight='bold',family='sans-serif')
    m.plot(x[0,:],  y[0,:],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[:,0],  y[:,0],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[-1,:], y[-1,:],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[:,-1], y[:,-1],linewidth=linefont, linestyle='solid', color='k')
    ax.spines['left'].set_linewidth(labelpadscale)
    ax.spines['top'].set_linewidth(labelpadscale)
    ax.spines['right'].set_linewidth(labelpadscale)
    ax.spines['bottom'].set_linewidth(labelpadscale)
    plt.tick_params(axis='x',which='major',bottom='on',left='on',top='on',right='on',length=25,width=2,colors='black',direction='out')       
    cbaxes1     = fig.add_axes([0.91, 0.595, 0.01, 0.22])
    cb1 = plt.colorbar(CB1,orientation='vertical',cax = cbaxes1)
    cb1.set_ticks(tick_marks1)
    cb1.ax.tick_params(labelsize=1.0*labelfont)
    cb1.set_label(r'ZVT trend (Sv per decade)', fontsize=1.0*labelfont,labelpad=10,family='sans-serif')
    cbaxes2     = fig.add_axes([0.91, 0.325, 0.01, 0.22])
    cb2 = plt.colorbar(CB1,orientation='vertical',cax = cbaxes2)
    cb2.set_ticks(tick_marks1)
    cb2.ax.tick_params(labelsize=1.0*labelfont)
    cb2.set_label(r'MVT trend (Sv per decade)', fontsize=1.0*labelfont,labelpad=10,family='sans-serif')
fig.savefig(directory+'/Figures/Figure3_uv_transport_trends.png',dpi=300,bbox_inches = 'tight')